In [2]:
import os, sys, errno
import argparse
import numpy as np
import keras
from misc import *
from interleave_convolutional import *
from dataset_utils import *
from gan_models import *

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
model_path = "/content/drive/MyDrive/Stuff I Coded/PCG_synthesis/out/model_g_present_450.h5"
norm_path = "/content/drive/MyDrive/Stuff I Coded/PCG_synthesis/out/normalization.npy"
out_dir = "/content/drive/MyDrive/Stuff I Coded/PCG_synthesis/generated"

In [8]:
MODEL = model_path   # file containing the weights of the model to load
NORM = norm_path     # file containing the normalization factors used
REPR = "dft"         # output type: dft, dct, or anything else for raw waveform
IL = True            # use ILConv to bulid model (always True for raw waveform)
BIAS = True          # generator output layer has a bias
O = out_dir          # output directory to place wav files
LSIZE = 100          # number of latent dimensions
MSIZE = 64           # model size
N = 10                # number of wav files to generate
B = 5                # number of wav files to generate per network run
R = 4000             # sample rate of output files
V = True             # output per-batch progress

In [6]:
# create and build model
norm_f = np.load(NORM)
if (REPR == 'dft'):
    gen_func = dft_generator if IL else dft_generator_tr
elif (IL == 'dct'):
    gen_func = dct_generator if IL else dct_generator_tr
else:
    gen_func = wave_generator
generator = gen_func(MSIZE, LSIZE, BIAS)
generator.load_weights(MODEL)

In [9]:
# generate audio
wav_i = 0
z_rs = np.random.RandomState(seed=177013)
while (wav_i < N):
    batch_size = min(B, N - wav_i)
    if (V):
        print('generating files {:d}-{:d} (out of {:d})...'.format(wav_i, wav_i + batch_size - 1, N))
    z_in = z_rs.uniform(low=-1, high=1, size=(batch_size, LSIZE)).astype(np.float32)
    G_z = generator.predict(z_in, batch_size=batch_size) / norm_f
    if (REPR == 'dft'):
        G_z = dft_transform_backward(G_z)
    elif (REPR == 'dct'):
        G_z = dct_transform_backward(G_z)
    else:
        G_z = np.squeeze(G_z)
    write_wav_dataset(G_z, O, fname_init=wav_i)
    wav_i += batch_size
if (V):
    print('done')

generating files 0-4 (out of 10)...
1/1 [==============================] - 0s 258ms/step
generating files 5-9 (out of 10)...
1/1 [==============================] - 0s 237ms/step
done
